In [10]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats as ss
import scipy.signal
from astropy.stats import bootstrap as bootstrap
from statsmodels.tsa.ar_model import AutoReg
import math
import common_func as cf
import import_ipynb
from Vogel import find_trend as ft
from check_ln2 import fit_dist, qq

ImportError: cannot import name 'qq' from 'check_ln2' (check_ln2.ipynb)

In [ ]:

# Function to fit data to a normal distribution, using either MLE or MOM
def fitNormal(data, method):
    assert method == 'MLE' or method == 'MOM', "method must = 'MLE' or 'MOM'"

    xbar, std, skew = cf.findmoments(data)

    if method == 'MLE':
        mu, sigma = ss.norm.fit(data)

    elif method == 'MOM':
        mu = xbar
        sigma = std

    return mu, sigma


#MannKendall Test

# function for determining tie groups and corresponding number of ties.
def count_ties(x):
    ties = []
    y = x
    while len(x) > 0: 
        ties.append(np.where(x == x[0])[0].size)
        x = np.delete(x, np.where(x == x[0]))
    ties=np.array(ties)
    return ties

#function for calculating variance in mann kendall test
def var(x):
    n = len(x)
    var = n * (n - 1) * (2 * n + 5)
    e = count_ties(x)
    extent_sum = np.sum(e * (e - 1) * (2 * e + 5))
    return  (var - extent_sum)/18

# function for testing pvalue and critical value of mann kendall test
def mktest(z):
    alpha = 0.1    # alpha = 0.05 
    CritVal = np.abs(ss.norm.ppf(alpha / 2))
    pvalue = ss.norm.sf(np.abs(z)) * 2 
    print('pvalue:',pvalue)
# Reject H0 if |Z-Score| > CritVal and, equivalently, if p-value < alpha/2
    reject1 = CritVal < np.abs(z)
    reject2 = pvalue < alpha
    if reject1:
        return 'Yes'
    return 'No'
# Mann Kendall Test
def main_mk(maxQ):
    T = 0
    data = np.log(maxQ)
    for j in range(len(data)-1):
        t = data[j] - (data[j+1:])
        T += (np.where(t < 0))[0].size - (np.where(t > 0))[0].size
    variance = var(data)
    if T > 0:
        z = (T - 1)/(np.sqrt(variance))
    else:
        z = (T + 1)/(np.sqrt(variance))
    print('Z_value:',z)
    trend = mktest(z)
    print('Is there a monotonic trend?',trend,'\n')

In [ ]:
# log transformation?
mu, sigma = fitNormal(np.log(maxQ),'MOM')
rho_log, p_value_log = NormalPPCT(np.log(maxQ), mu, sigma, 'Log Normal PPCT', 'LogNormalPPCT.png')
print(rho_log, p_value_log)

In [ ]:
def acf_pacf(maxQ):
    sm.graphics.tsa.plot_acf(np.log(maxQ))
    plt.show()
    sm.graphics.tsa.plot_pacf(np.log(maxQ))
    plt.show()

In [ ]:
Data = pd.read_csv('precip.csv')
maxQ, yearMaxQ = cf.annual_max(Data)
# Get just the year vector for annual maxima as numpy array
fig = plt.figure()
plt.plot(yearMaxQ, maxQ)
plt.xlabel('Year', fontsize=16)
plt.ylabel('Precipitation', fontsize=16)
plt.title('Annual Precipitation', fontsize=16)
fig.set_size_inches([9, 7.25])
fig.savefig('Annual Precipitation', dpi=300)
plt.show()



